In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium import plugins
#import seaborn as sns
%matplotlib inline
from folium.plugins import HeatMap
import os
import random
import urllib3
import json
pd.options.mode.chained_assignment = None

In [2]:
shootings = pd.read_csv('NYPD_Shooting_Incident_Data.csv')
nycSoc = pd.read_excel('soc_2016acs5yr_nta.xlsx')
nycDemo = pd.read_excel('demo_2016acs5yr_nta.xlsx')
ntaToDist = pd.read_csv('https://gist.githubusercontent.com/spmich/1c52c5cca96c3d4ea9c69149def1bbe8/raw/1f6ee5c552928981de63e5a74547cd3593258565/gistfile1.txt')

In [3]:
victimShootings = shootings[['OCCUR_DATE','OCCUR_TIME','BORO','PRECINCT','VIC_AGE_GROUP','VIC_SEX','VIC_RACE','X_COORD_CD','Y_COORD_CD','Latitude','Longitude']]
prepShootings = shootings[['OCCUR_DATE','OCCUR_TIME','BORO','PRECINCT','PERP_AGE_GROUP','PERP_SEX','PERP_RACE','X_COORD_CD','Y_COORD_CD','Latitude','Longitude']]

In [4]:
#victimShootings.head()

In [5]:
victimShootings['OCCUR_DATE'] = pd.to_datetime(victimShootings['OCCUR_DATE'])
victimShootings['Year'] = victimShootings['OCCUR_DATE'].dt.year
victimShootings.to_csv(r'C:\Users\LenovoPC1\Documents\Social Data Analysis\Finale Project\victimShootings.csv', index = False)

In [6]:
victimShootings.to_json(r'C:\Users\LenovoPC1\Documents\Social Data Analysis\Finale Project\victimShootings.json')

In [7]:
#victimShootings.groupby(['PRECINCT','Year'])
#victimShootings.groupby(['PRECINCT', 'Year']).apply(lambda x: x.to_json(r'C:\Users\LenovoPC1\Documents\Social Data Analysis\Finale Project\victimShootings.json'))

In [8]:
prepShootings = prepShootings.dropna(how='any')
prepShootings = prepShootings[prepShootings['PERP_RACE'] != 'UNKNOWN']
prepShootings = prepShootings[prepShootings['PERP_AGE_GROUP'] != 'UNKNOWN']
prepShootings = prepShootings[prepShootings['PERP_SEX'] != 'UNKNOWN']
prepShootings.to_csv(r'C:\Users\LenovoPC1\Documents\Social Data Analysis\Finale Project\prepShootings.csv', index = False)
#prepShootings.head()

In [9]:
nycSoc.head()

,GeoType,GeogName,GeoID,Borough,HH1E,HH1M,HH1C,HH1P,HH1Z,Fam1E,...,OthrE,OthrM,OthrC,OthrP,OthrZ,UnclsNRE,UnclsNRM,UnclsNRC,UnclsNRP,UnclsNRZ
0,NTA2010,Bath Beach,BK27,Brooklyn,11428,209.0,1.1,100.0,NaN,8207,...,15051,1186.0,4.8,45.4,3.1,6697,1100.0,10.0,20.2,3.2
1,NTA2010,Bay Ridge,BK31,Brooklyn,33487,442.0,0.8,100.0,NaN,18935,...,27814,1530.0,3.3,35.1,1.8,7423,806.0,6.6,9.4,1.0
2,NTA2010,Bedford,BK75,Brooklyn,26059,401.0,0.9,100.0,NaN,14540,...,41891,1898.0,2.8,55.6,2.0,9693,1035.0,6.5,12.9,1.3
3,NTA2010,Bensonhurst East,BK29,Brooklyn,22187,338.0,0.9,100.0,NaN,16276,...,34121,1732.0,3.1,53.1,2.3,9077,1007.0,6.7,14.1,1.5
4,NTA2010,Bensonhurst West,BK28,Brooklyn,29909,364.0,0.7,100.0,NaN,21832,...,47340,1974.0,2.5,51.7,1.8,15409,1377.0,5.4,16.8,1.5


In [10]:
filteredSoc = nycSoc[['GeogName','GeoID','Borough','EA_LT9GE','EA_9t12NDE','EA_HScGrdE','EA_SClgNDE','EA_AscDE','EA_BchDE','EA_GrdPfDE','EA_LTHSGrE','EA_BchDHE']]

In [11]:
filteredSoc = filteredSoc.rename(index=str, columns={"EA_LT9GE": "Less than 9th grade", 
                                       "EA_9t12NDE": "9th to 12th grade", 
                                       'EA_HScGrdE' : "High School",
                                       'EA_SClgNDE' : "Some college, no degree",
                                       'EA_AscDE' : "Associate degree",
                                       'EA_BchDE' : "Bachelor degree", 
                                       'EA_GrdPfDE' : "Graduate or proffesional degree", 
                                       'EA_LTHSGrE' : "Less than High school", 
                                       'EA_BchDHE' : "Bachelor degree or higher"})

In [12]:
filteredSoc.head()

,GeogName,GeoID,Borough,Less than 9th grade,9th to 12th grade,High School,"Some college, no degree",Associate degree,Bachelor degree,Graduate or proffesional degree,Less than High school,Bachelor degree or higher
0,Bath Beach,BK27,Brooklyn,3146,2326,6188,2486,1667,5345,2664,5472,8009
1,Bay Ridge,BK31,Brooklyn,4106,3900,11165,8251,3746,15823,11187,8006,27010
2,Bedford,BK75,Brooklyn,3908,5791,11608,6838,2215,9148,5258,9699,14406
3,Bensonhurst East,BK29,Brooklyn,6856,4700,12871,4433,3262,9412,4476,11556,13888
4,Bensonhurst West,BK28,Brooklyn,11095,7191,18127,6748,4796,12275,4784,18286,17059


In [13]:
filteredSoc.to_csv(r'C:\Users\LenovoPC1\Documents\Social Data Analysis\Finale Project\DegreeNYC.csv', index = False)

In [14]:
#filteredShootings['VIC_RACE'].unique()

In [15]:
nycPop = nycDemo[['GeogName','GeoID','Borough','Pop_2E']]

In [16]:
nycPop = nycPop.rename(index=str, columns={'Pop_2E' : 'NTA_Population'})

In [17]:
nycPop.to_csv(r'C:\Users\LenovoPC1\Documents\Social Data Analysis\Finale Project\PopulationNYC.csv', index = False)

In [18]:
#districtData = pd.DataFrame(columns=['Precinct', 'Total Population', 'White'])
#for index, row in ntaToDist.iterrows():
#    for index2, row2 in filteredSoc.iterrows():
#        if row["NTAName"] == row2["GeogName"]:
#            print(districtData["Precinct"])

In [19]:
#prepShootings.groupby("PERP_RACE").size()

In [20]:
#victimShootings.groupby('VIC_RACE').size()

In [21]:
#prepShootings.count()